### Testing for Interactive use case

In [2]:
import mlflow
from azureml.core import Workspace, Experiment, Environment, Datastore, Dataset, ScriptRunConfig
from azureml.core.runconfig import PyTorchConfiguration
# from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import PyTorchConfiguration
from azureml.core.environment import Environment
from azureml.core.conda_dependencies import CondaDependencies
from IPython.display import clear_output
import time
import platform
# from ray_on_azureml.ray_on_aml import getRay
import sys
sys.path.append("../") # go to parent dir
import importlib


In [1]:
# pip install --upgrade ray[default]

In [ ]:
pip install raydp

In [ ]:
# pip install raydp /azureml-envs/azureml_6cb52194be4f7fc697297312b8f55547/lib/python3.8/site-packages/ray/jars/ray_dist.jar

In [3]:
base_image="FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1"
dockerfile = r"""
{0}
ARG HTTP_PROXY
ARG HTTPS_PROXY
# set http_proxy & https_proxy
ENV http_proxy=${{HTTPS_PROXY}}
ENV https_proxy=${{HTTPS_PROXY}}
RUN http_proxy=${{HTTPS_PROXY}} https_proxy=${{HTTPS_PROXY}} apt-get update -y \
    && mkdir -p /usr/share/man/man1 \
    && http_proxy=${{HTTPS_PROXY}} https_proxy=${{HTTPS_PROXY}} apt-get install -y openjdk-11-jdk \
    && mkdir /raydp \
    && pip --no-cache-dir install raydp
WORKDIR /raydp
# unset http_proxy & https_proxy
ENV http_proxy=
ENV https_proxy=
""".format(base_image)
print(dockerfile)


FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1
ARG HTTP_PROXY
ARG HTTPS_PROXY
# set http_proxy & https_proxy
ENV http_proxy=${HTTPS_PROXY}
ENV https_proxy=${HTTPS_PROXY}
RUN http_proxy=${HTTPS_PROXY} https_proxy=${HTTPS_PROXY} apt-get update -y \
    && mkdir -p /usr/share/man/man1 \
    && http_proxy=${HTTPS_PROXY} https_proxy=${HTTPS_PROXY} apt-get install -y openjdk-11-jdk \
    && mkdir /raydp \
    && pip --no-cache-dir install raydp
WORKDIR /raydp
# unset http_proxy & https_proxy
ENV http_proxy=
ENV https_proxy=



In [3]:
from src.ray_on_aml.core import Ray_On_AML
# from ray_on_aml.core import Ray_On_AML

ws = Workspace.from_config()
ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="d15-v2", additional_pip_packages=['torch==1.10.0', 'torchvision', 'sklearn'], maxnode=2)
# ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="gpunc6", base_pip_dep = ['ray[tune]==1.9.2', 'ray[rllib]==1.9.2','ray[serve]==1.9.2', 'xgboost_ray==0.1.6', 'dask==2021.12.0','pyarrow >= 5.0.0','fsspec==2021.10.1','fastparquet==0.7.2','tabulate==0.8.9','raydp'], maxnode=2)

# ray_on_aml =Ray_On_AML(ws=ws, compute_cluster ="worker-cpu-v3", base_pip_dep = ['ray[default]==1.8.0', 'xgboost_ray==0.1.6', 'dask==2021.12.0',\
# 'pyarrow >= 5.0.0','fsspec==2021.10.1','fastparquet==0.7.2','tabulate==0.8.9','pyspark'], maxnode=2)
ray = ray_on_aml.getRay()

# ray = ray_on_aml.getRay(gpu_support=True)
ray.cluster_resources()

Cancel active AML runs if any
Canceling active run  ray_on_aml_1644811136_d1923402 in ray_on_aml
Shutting down ray if any
Found existing cluster d15-v2
Creating new Environment ray-0.0.7-1634135180135722568
Waiting for cluster to start


{'accelerator_type:K80': 1.0,
 'node:10.0.0.13': 1.0,
 'CPU': 6.0,
 'GPU': 1.0,
 'object_store_memory': 16625394892.0,
 'memory': 33250789787.0}

In [23]:
ray.cluster_resources()

{'accelerator_type:K80': 1.0,
 'node:10.0.0.13': 1.0,
 'memory': 238150763316.0,
 'GPU': 1.0,
 'object_store_memory': 104381099212.0,
 'CPU': 46.0,
 'node:10.0.0.5': 1.0,
 'node:10.0.0.21': 1.0}

In [ ]:
# You can run this code outside of the Ray cluster!
import ray
ray.__version__

In [ ]:
conda_lib_path = sys.executable.split('/')[-3]+"/lib/python"+sys.version[:3]
conda_lib_path

In [ ]:
pip install --upgrade pyspark

In [ ]:
!ls /anaconda/envs/azureml_py38/lib/python3.8/site-packages/pyspark/jars/

In [ ]:
ray.cluster_resources()

### Testing with Dask on Ray

In [4]:
from adlfs import AzureBlobFileSystem

abfs = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")
#if read all years and months
# data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather//", filesystem=abfs)
data =ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2015/"], filesystem=abfs)

Metadata Fetch Progress: 100%|██████████| 16/16 [00:07<00:00,  2.09it/s]


In [6]:
#Scaling up date with Dask dataframe API
import dask
from ray.util.dask import ray_dask_get,enable_dask_on_ray
enable_dask_on_ray()
import pandas as pd
import numpy as np
import dask.dataframe as dd

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2015/puMonth=*/*.parquet', storage_options=storage_options)
ddf.count().compute()


# Set the scheduler to ray_dask_get in your config so you don't have to
# specify it on each compute call.

df = dd.from_pandas(
    pd.DataFrame(
        np.random.randint(0, 10000, size=(1024, 2)), columns=["age", "grade"]),
    npartitions=2)
df.groupby(["age"]).mean().compute()

# ray.shutdown()

2022-02-14 04:56:32,313	ERROR serialization.py:289 -- Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 287, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 194, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 172, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 160, in _deserialize_pickle

RaySystemError: System error: Can't get attribute '_unpickle_block' on <module 'pandas._libs.internals' from '/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/pandas/_libs/internals.cpython-38-x86_64-linux-gnu.so'>
traceback: Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 287, in deserialize_objects
    obj = self._deserialize_object(data, metadata, object_ref)
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 194, in _deserialize_object
    return self._deserialize_msgpack_data(data, metadata_fields)
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 172, in _deserialize_msgpack_data
    python_objects = self._deserialize_pickle5_data(pickle5_data)
  File "/anaconda/envs/azureml_py38_pytorch/lib/python3.8/site-packages/ray/serialization.py", line 160, in _deserialize_pickle5_data
    ob

In [ ]:
import dask.dataframe as dd

storage_options = {'account_name': 'azureopendatastorage'}
ddf = dd.read_parquet('az://nyctlc/green/puYear=2019/puMonth=*/*.parquet', storage_options=storage_options)
ddf.count().compute()

In [ ]:
#dask

# import ray
from ray.util.dask import ray_dask_get
import dask.array as da
import dask.dataframe as dd
import numpy as np
import pandas as pd
import dask.dataframe as dd
import matplotlib.pyplot as plt

from datetime import datetime

from azureml.core import Workspace, Dataset, Model
from adlfs import AzureBlobFileSystem
account_key = ws.get_default_keyvault().get_secret("adls7-account-key")
account_name="adlsgen7"
abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


storage_options={'account_name': account_name, 'account_key': account_key}

# ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)

data = ray.data.read_parquet("az://isdweatherdatacontainer/ISDWeather/year=2009", filesystem=abfs2)
data2 = ray.data.read_parquet("az://mltraining/ISDWeatherDelta/year2008", filesystem=abfs)
data.count()

In [ ]:


from datetime import datetime

from azureml.core import Workspace, Dataset, Model
from adlfs import AzureBlobFileSystem
account_key = ws.get_default_keyvault().get_secret("adls7-account-key")
account_name="adlsgen7"
# abfs = AzureBlobFileSystem(account_name="adlsgen7",account_key=account_key,  container_name="mltraining")
abfs2 = AzureBlobFileSystem(account_name="azureopendatastorage",  container_name="isdweatherdatacontainer")


storage_options={'account_name': account_name, 'account_key': account_key}

# ddf = dd.read_parquet('az://mltraining/ISDWeatherDelta/year2008', storage_options=storage_options)

data = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2012/"], filesystem=abfs2)
data1 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2015/"], filesystem=abfs2)
data2 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2010/"], filesystem=abfs2)
data3 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2009/"], filesystem=abfs2)
data4 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2011/"], filesystem=abfs2)
data5 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2013/"], filesystem=abfs2)
data6 = ray.data.read_parquet(["az://isdweatherdatacontainer/ISDWeather/year=2014/"], filesystem=abfs2)
all_data =data.union(data1).union(data2).union(data3).union(data4).union(data5).union(data6)
all_data.count()
start = time.time()
#convert Ray dataset to Dask dataframe 
all_data_dask = data.to_dask().describe().compute()
print(all_data_dask)
stop = time.time()
print("duration ", (stop-start))
#717s for single machine nc6
# duration  307.69699811935425s for CI as head and 4 workers of DS14_v2

### Testing Ray Tune for distributed ML tunning

In [12]:
import numpy as np
import torch
import torch.optim as optim
import torch.nn as nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import torch.nn.functional as F
# import ray
from ray import tune
from ray.tune.schedulers import ASHAScheduler
class ConvNet(nn.Module):
    def __init__(self):
        super(ConvNet, self).__init__()
        # In this example, we don't change the model architecture
        # due to simplicity.
        self.conv1 = nn.Conv2d(1, 3, kernel_size=3)
        self.fc = nn.Linear(192, 10)

    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1(x), 3))
        x = x.view(-1, 192)
        x = self.fc(x)
        return F.log_softmax(x, dim=1)
# Change these values if you want the training to run quicker or slower.
EPOCH_SIZE = 512
TEST_SIZE = 256

def train(model, optimizer, train_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        # We set this just for the example to run quickly.
        if batch_idx * len(data) > EPOCH_SIZE:
            return
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()


def test(model, data_loader):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for batch_idx, (data, target) in enumerate(data_loader):
            # We set this just for the example to run quickly.
            if batch_idx * len(data) > TEST_SIZE:
                break
            data, target = data.to(device), target.to(device)
            outputs = model(data)
            _, predicted = torch.max(outputs.data, 1)
            total += target.size(0)
            correct += (predicted == target).sum().item()

    return correct / total
def train_mnist(config):
    # Data Setup
    mnist_transforms = transforms.Compose(
        [transforms.ToTensor(),
         transforms.Normalize((0.1307, ), (0.3081, ))])

    train_loader = DataLoader(
        datasets.MNIST("~/data", train=True, download=True, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)
    test_loader = DataLoader(
        datasets.MNIST("~/data", train=False, transform=mnist_transforms),
        batch_size=64,
        shuffle=True)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = ConvNet()
    model.to(device)

    optimizer = optim.SGD(
        model.parameters(), lr=config["lr"], momentum=config["momentum"])
    for i in range(10):
        train(model, optimizer, train_loader)
        acc = test(model, test_loader)

        # Send the current training result back to Tune
        tune.report(mean_accuracy=acc)

        if i % 5 == 0:
            # This saves the model to the trial directory
            torch.save(model.state_dict(), "./model.pth")
search_space = {
    "lr": tune.sample_from(lambda spec: 10**(-10 * np.random.rand())),
    "momentum": tune.uniform(0.01, 0.09)
}

# Uncomment this to enable distributed execution
# ray.shutdown()
# ray.init(address="auto",ignore_reinit_error=True)
# ray.init(address =f'ray://{headnode_private_ip}:10001',allow_multiple=True,ignore_reinit_error=True )
# Download the dataset first
datasets.MNIST("~/data", train=True, download=True)

analysis = tune.run(train_mnist, config=search_space)


  0%|          | 0/9912422 [00:00<?, ?it/s]

Extracting /home/azureuser/data/MNIST/raw/train-images-idx3-ubyte.gz to /home/azureuser/data/MNIST/raw



  0%|          | 0/28881 [00:00<?, ?it/s]

Extracting /home/azureuser/data/MNIST/raw/train-labels-idx1-ubyte.gz to /home/azureuser/data/MNIST/raw



  0%|          | 0/1648877 [00:00<?, ?it/s]

...

Trial name,status,loc,lr,momentum
train_mnist_f81e8_00000,PENDING,,1.22836e-09,0.0819143


2022-01-30 03:59:07,287	ERROR trial_runner.py:958 -- Trial train_mnist_f81e8_00000: Error processing event.
Traceback (most recent call last):
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/tune/trial_runner.py", line 924, in _process_trial
    results = self.trial_executor.fetch_result(trial)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/tune/ray_trial_executor.py", line 787, in fetch_result
    result = ray.get(trial_future[0], timeout=DEFAULT_GET_TIMEOUT)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/_private/client_mode_hook.py", line 105, in wrapper
    return func(*args, **kwargs)
  File "/anaconda/envs/azureml_py38/lib/python3.8/site-packages/ray/worker.py", line 1715, in get
    raise value
ray.exceptions.RayActorError: The actor died because of an error raised in its creation task, ray::ImplicitFunc.__init__() (pid=208, ip=10.0.0.18)
RuntimeError: The actor with name ImplicitFunc failed to import on the worker. T

Result for train_mnist_f81e8_00000:
  trial_id: f81e8_00000
  


(TemporaryActor pid=208, ip=10.0.0.18) 2022-01-30 03:59:07,297	ERROR worker.py:431 -- Exception raised in creation task: The actor died because of an error raised in its creation task, ray::ImplicitFunc.__init__() (pid=208, ip=10.0.0.18)
(TemporaryActor pid=208, ip=10.0.0.18) RuntimeError: The actor with name ImplicitFunc failed to import on the worker. This may be because needed library dependencies are not installed in the worker environment:
(TemporaryActor pid=208, ip=10.0.0.18) 
(TemporaryActor pid=208, ip=10.0.0.18) ray::ImplicitFunc.__init__() (pid=208, ip=10.0.0.18)
(TemporaryActor pid=208, ip=10.0.0.18)   File "/azureml-envs/azureml_b754358d546bd4cc08fdd901eb6aa8f0/lib/python3.8/site-packages/ray/cloudpickle/cloudpickle.py", line 679, in subimport
(TemporaryActor pid=208, ip=10.0.0.18)     __import__(name)
(TemporaryActor pid=208, ip=10.0.0.18) ModuleNotFoundError: No module named 'torchvision'


TuneError: ('Trials did not complete', [train_mnist_f81e8_00000])

In [ ]:
 import sklearn.datasets
 import sklearn.metrics
 from sklearn.model_selection import train_test_split
 import xgboost as xgb

 from ray import tune


 def train_breast_cancer(config):
     # Load dataset
     data, labels = sklearn.datasets.load_breast_cancer(return_X_y=True)
     # Split into train and test set
     train_x, test_x, train_y, test_y = train_test_split(
         data, labels, test_size=0.25)
     # Build input matrices for XGBoost
     train_set = xgb.DMatrix(train_x, label=train_y)
     test_set = xgb.DMatrix(test_x, label=test_y)
     # Train the classifier
     results = {}
     xgb.train(
         config,
         train_set,
         evals=[(test_set, "eval")],
         evals_result=results,
         verbose_eval=False)
     # Return prediction accuracy
     accuracy = 1. - results["eval"]["error"][-1]
     tune.report(mean_accuracy=accuracy, done=True)


 config = {
     "objective": "binary:logistic",
     "eval_metric": ["logloss", "error"],
     "max_depth": tune.randint(1, 9),
     "min_child_weight": tune.choice([1, 2, 3]),
     "subsample": tune.uniform(0.5, 1.0),
     "eta": tune.loguniform(1e-4, 1e-1)
 }
 analysis = tune.run(
     train_breast_cancer,
     resources_per_trial={"cpu": 1},
     config=config,
     num_samples=20)


In [ ]:
pip install raydp-nightly


In [ ]:
import subprocess
python_path = subprocess.check_output("which python", shell=True).strip()
python_path = python_path.decode('utf-8')
python_path = python_path +"/site-packages/raydp/jars/raydp-0.5.0-SNAPSHOT.jar"
os.listdir('/anaconda/envs/azureml_py38/bin//python/site-packages/raydp/jars')

### Testing Spark on Ray

In [ ]:
os.copy(/anaconda/envs/azureml_py38/lib/python3.8/site-packages/raydp/jars/raydp-0.5.0-SNAPSHOT.jar

In [ ]:
import raydp
os.path.abspath(os.path.join(os.path.abspath(raydp.__file__), "../../jars/*"))

In [ ]:
!ls /anaconda/envs/azureml_py38/lib/python3.8/site-packages/jars/*

In [ ]:
# RAYDP_CP = os.path.abspath(os.path.join(os.path.abspath(__file__), "../../jars/*"))
import ray
RAY_CP = os.path.abspath(os.path.join(os.path.dirname(ray.__file__), "jars/*"))
RAY_CP

In [ ]:
from azureml.core import Workspace, Experiment, Environment,ScriptRunConfig
from azureml.widgets import RunDetails
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.core.runconfig import RunConfiguration
#Remember the AML job has to have distribted setings (MPI type) for ray-on-aml to work correctly.
ws = Workspace.from_config()
compute_cluster = 'd12-ssh-novnet' #This can be another cluster different from the interactive cluster. 
ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

aml_run_config_ml = RunConfiguration(communicator='OpenMpi')


#Check the conda_env.yml, it has an entry of ray-on-aml
rayEnv = Environment.from_conda_specification(name = "RLEnv",
                                             file_path = "conda_env.yml")
dockerfile = r"""
FROM mcr.microsoft.com/azureml/openmpi3.1.2-ubuntu18.04:20210615.v1
ARG HTTP_PROXY
ARG HTTPS_PROXY
# set http_proxy & https_proxy
ENV http_proxy=${HTTP_PROXY}
ENV https_proxy=${HTTPS_PROXY}
RUN http_proxy=${HTTP_PROXY} https_proxy=${HTTPS_PROXY} apt-get update -y \
    && mkdir -p /usr/share/man/man1 \
    && http_proxy=${HTTP_PROXY} https_proxy=${HTTPS_PROXY} apt-get install -y openjdk-11-jdk \
    && mkdir /raydp \
    && pip --no-cache-dir install raydp
WORKDIR /raydp
# unset http_proxy & https_proxy
ENV http_proxy=
ENV https_proxy=
"""

# Set the base image to None, because the image is defined by Dockerfile.
rayEnv.docker.base_image = None
rayEnv.docker.base_dockerfile = dockerfile

aml_run_config_ml.target = ray_cluster
aml_run_config_ml.node_count = 3
aml_run_config_ml.environment = rayEnv
src = ScriptRunConfig(source_directory='job',
                    script='aml_job.py',
                    run_config = aml_run_config_ml,
                   )

run = Experiment(ws, "rl_on_aml_job").submit(src)


In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()


In [ ]:
run.cancel()

In [ ]:
import raydp
raydp.__version__

In [ ]:


additional_spark_configs ={"fs.azure.account.key.adlsdatalakegen6.blob.core.windows.net":"AcDil/MwM9KlDvJu0LBcBIQxogAncv306NMRYABtjphXfWgaDTV3yjZgoSNckUb/3nhG04ND2Nqn553fq36Pqw==",
          "fs.azure.account.key.adlsdatalakegen6.dfs.core.windows.net":"AcDil/MwM9KlDvJu0LBcBIQxogAncv306NMRYABtjphXfWgaDTV3yjZgoSNckUb/3nhG04ND2Nqn553fq36Pqw=="}
other_configs = {"raydp.executor.extraClassPath":"/azureml-envs/azureml_6cb52194be4f7fc697297312b8f55547/lib/python3.8/site-packages/pyspark/jars/*"}
spark = ray_on_aml.getSpark(executor_cores =3,num_executors =3 ,executor_memory='10GB', additional_spark_configs=additional_spark_configs)



22:55:41.408 [dispatcher-CoarseGrainedScheduler] ERROR org.apache.spark.scheduler.TaskSchedulerImpl - Lost executor 2 on 10.0.0.5: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


2022-01-29 22:55:45,516	ERROR worker.py:478 -- print_logs: Connection closed by server.
2022-01-29 22:55:45,517	ERROR worker.py:1247 -- listen_error_messages_raylet: Connection closed by server.
2022-01-29 22:55:45,520	ERROR import_thread.py:89 -- ImportThread: Connection closed by server.


22:55:45.453 [dispatcher-CoarseGrainedScheduler] ERROR org.apache.spark.scheduler.TaskSchedulerImpl - Lost executor 0 on 10.0.0.13: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.
22:55:45.457 [dispatcher-CoarseGrainedScheduler] ERROR org.apache.spark.scheduler.TaskSchedulerImpl - Lost executor 1 on 10.0.0.20: Remote RPC client disassociated. Likely due to containers exceeding thresholds, or network issues. Check driver logs for WARN messages.


[2022-01-29 23:05:44,751 E 15477 15529] gcs_server_address_updater.cc:76: Failed to receive the GCS address for 600 times without success. The worker will exit ungracefully. It is because GCS has died. It could be because there was an issue that kills GCS, such as high memory usage triggering OOM killer to kill GCS. Cluster will be highly likely unavailable if you see this log. Please check the log from gcs_server.err.


In [ ]:
# ray.cluster_resources()

In [ ]:
# import ray
# import raydp
# # # # import os


# # # ray.init(address ='ray://10.0.0.11:6379')
# spark = raydp.init_spark(
#   app_name = "example",
#   num_executors = 2,
#   executor_cores = 2,
#   executor_memory = "1GB",
# #     configs = {"spark.jars":"jars/*.jar"}
#     configs = {"spark.jars":"jars/azure-storage-8.6.6.jar,jars/hadoop-azure-3.3.1.jar,jars/jetty-util-ajax-11.0.7.jar,jars/jetty-util-9.3.24.v20180605.jar,jars/delta-core_2.12-1.1.0.jar,jars/mssql-jdbc-9.4.1.jre8.jar",
#               "fs.azure.account.key.adlsdatalakegen6.blob.core.windows.net":"AcDil/MwM9KlDvJu0LBcBIQxogAncv306NMRYABtjphXfWgaDTV3yjZgoSNckUb/3nhG04ND2Nqn553fq36Pqw==",
#               "fs.azure.account.key.adlsdatalakegen6.dfs.core.windows.net":"AcDil/MwM9KlDvJu0LBcBIQxogAncv306NMRYABtjphXfWgaDTV3yjZgoSNckUb/3nhG04ND2Nqn553fq36Pqw=="}

# #     configs = {"spark.jars":"jars/mssql-jdbc-9.4.1.jre8.jar"}
# #     configs = {
# #                "spark.driver.extraClassPath":"jars/mssql-jdbc-9.4.1.jre8.jar"}

# #     configs = {"spark.jars.packages":"org.apache.hadoop:hadoop-azure:3.3.1,com.microsoft.azure:azure-storage:8.6.6"}
# )




In [ ]:
import raydp
raydp.stop_spark()

In [ ]:
adls_data = spark.read.format("delta").load("wasbs://mltraining@adlsdatalakegen6.blob.core.windows.net/ISDWeatherDelta")

In [ ]:
adls_data = spark.read.format("delta").load("abfss://mltraining@adlsdatalakegen6.dfs.core.windows.net/ISDWeatherDelta")

In [ ]:
adls_data.groupby("stationName").count().head(20)
# 73,696,631

In [ ]:
ray.cluster_resources()

In [ ]:
adls_data.write.format("delta").mode("overwrite").save("wasbs://mltraining@adlsdatalakegen6.blob.core.windows.net/FirstRaySave")

In [ ]:
adls_data.selectExpr("snowDepth","stationName")

In [ ]:
ray_data = ray.data.from_spark(data)

#### Synapse SQL Pool Data Access

In [ ]:
server_name = "jdbc:sqlserver://sy2qwhqqkv7eacsws1.sql.azuresynapse.net:1433"
database_name = "sy2qwhqqkv7eacsws1p1"
url = server_name + ";" + "databaseName=" + database_name + ";"

table_name = "ISDWeatherDelta"
username = "azureuser"
password = "abcd@12345" # Please specify password here
# try:
#     adls_data.write \
#     .format("com.microsoft.sqlserver.jdbc.spark") \
#     .mode("overwrite") \
#     .option("url", url) \
#     .option("dbtable", table_name) \
#     .option("user", username) \
#     .option("password", password) \
#     .save()
# except ValueError as error :
#     print("Connector write failed", error)

    

jdbcDF = spark.read \
        .format("jdbc") \
        .option("url", url) \
        .option("dbtable", table_name) \
        .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
        .option("user", username) \
        .option("password", password).load()


# sparkdf = spark.createDataFrame([(1, 2),(2, 9),(3, 7)],
#                             ("id", "name")) 
# try:
#     adls_data.selectExpr("snowDepth","latitude").write \
#     .format("jdbc") \
#     .mode("overwrite") \
#     .option("url", url) \
#     .option("driver", "com.microsoft.sqlserver.jdbc.SQLServerDriver") \
#     .option("dbtable", table_name) \
#     .option("user", username) \
#     .option("batchsize", 10000) \
#     .option("password", password) \
#     .save()
# except ValueError as error :
#     print("Connector write failed", error)

In [ ]:
import pandas as pd
from pyspark.sql.functions import col, pandas_udf
from pyspark.sql.types import LongType

def multiply_func(a: pd.Series, b: pd.Series) -> pd.Series:
    return a * b

multiply = pandas_udf(multiply_func, returnType=LongType())

# The function for a pandas_udf should be able to execute with local Pandas data
x = pd.Series([1, 2, 3])
df = spark.createDataFrame(pd.DataFrame(x, columns=["x"]))

# Execute function as a Spark vectorized UDF
df.select(multiply(col("x"), col("x"))).show()

In [ ]:
df2.write.format("delta").mode("overwrite").save("data/delta_test")

In [ ]:
import os
import re
import pandas as pd, numpy as np

from pyspark.sql.functions import *

from tensorflow import keras

import raydp
from raydp.tf import TFEstimator
from raydp.utils import random_split

In [ ]:
def fill_na(data):
    
    # Fill NA in column Fare, Age and Embarked
    data = data.fillna({"Embarked": "S"})
    
    fare_avg = data.select(mean(col("Fare")).alias("mean")).collect()
    data = data.na.fill({"Fare": fare_avg[0]["mean"]})
    
    age_avg = data.select(mean(col("Age")).alias("mean")).collect()
    data = data.na.fill({'Age': age_avg[0]["mean"]})
    
    return data


In [ ]:
def do_features(data):
    
    # Add some new features
    data = data.withColumn("name_length", length("Name"))
    data = data.withColumn("has_cabin", col("Cabin").isNotNull().cast('int'))
    
    data = data.withColumn("family_size", col("SibSp") + col("Parch") + 1)
    data = data.withColumn("is_alone", (col("family_size") == 1).cast('int'))
    
    
    # Add some features about passengers' title with spark udf
    @udf("string")
    def get_title(name):
        title = ''
        title_match = re.search(' ([A-Za-z]+)\.', name)
        if (title_match):
            title = title_match.group(1)
            if (title in ['Lady', 'Countess','Capt', 'Col','Don', 'Dr',
                          'Major', 'Rev', 'Sir', 'Jonkheer', 'Dona']):
                title = 'Rare'
            return title
        return title
    data = data.withColumn("Title", get_title(col("Name")))
    data = data.withColumn("Title", regexp_replace("Title", "Mlle|Ms", "Miss"))
    data = data.withColumn("Title", regexp_replace("Title", "Mme", "Mrs"))
    
    # Encode column Sex
    sex_udf = udf(lambda x: 0 if x == "female" else 1)
    data = data.withColumn("Sex", sex_udf(col("Sex")).cast('int'))
    
    # Encode column Title
    title_map = {"Mr": 1, "Miss": 2, "Mrs": 3, "Master": 4, "Rare": 5}
    title_udf = udf(lambda x: title_map[x])
    data = data.withColumn("Title", title_udf(col("Title")).cast('int'))
    
    # Encode column Embarked
    embarked_map = {'S': 0, 'C': 1, 'Q': 2}
    embarked_udf = udf(lambda x: embarked_map[x])
    data = data.withColumn("Embarked", embarked_udf(col("Embarked")).cast('int'))
    
    # Categorize column Fare
    @udf("int")
    def fare_map(fare):
        if (fare <= 7.91):
            return 0
        elif fare <= 14.454:
            return 1
        elif fare <= 31:
            return 2
        else:
            return 3
    data = data.withColumn("Fare", fare_map(col("Fare")))
    
    # Categorize column Age
    @udf("int")
    def age_map(age):
        if age <= 16:
            return 0
        elif age <= 32:
            return 1
        elif age <= 48:
            return 2
        elif age <= 64:
            return 3
        else:
            return 4
    data = data.withColumn("Age", age_map(col("Age")))
    
    return data
def drop_cols(data):
    
    # Drop useless columns
    data = data.drop("PassengerId") \
        .drop("Name") \
        .drop("Ticket") \
        .drop("Cabin") \
        .drop("SibSp")
    
    return data


In [ ]:
train = fill_na(train)

train = do_features(train)

train = drop_cols(train)


In [ ]:
train.show(5)

In [ ]:
features = [field.name for field in list(train.schema) if field.name != "Survived"]


In [ ]:
inTensor = []
for _ in range(len(features)):
    inTensor.append(keras.Input((1,)))
concatenated = keras.layers.concatenate(inTensor)
fc1 = keras.layers.Dense(32, activation='relu')(concatenated)
fc2 = keras.layers.Dense(32, activation='relu')(fc1)
dp1 = keras.layers.Dropout(0.25)(fc2)
fc3 = keras.layers.Dense(16, activation='relu')(dp1)
dp2 = keras.layers.Dropout(0.25)(fc3)
fc4 = keras.layers.Dense(1, activation='sigmoid')(dp2)
model = keras.models.Model(inTensor, fc4)


In [ ]:
rmsp = keras.optimizers.RMSprop()
loss = keras.losses.BinaryCrossentropy()
estimator = TFEstimator(num_workers=3, model=model, optimizer=rmsp, loss=loss, metrics=["binary_accuracy"],
                        feature_columns=features, label_column="Survived", batch_size=32, num_epochs=100,
                        config={"fit_config": {"steps_per_epoch": train.count() // 32}})


In [ ]:
estimator.fit_on_spark(train, None)


In [ ]:
raydp.stop_spark()


## Testing Ray on Job Cluster

In [ ]:
#     pyarrow >=6.0.1
# dask >=2021.11.2
# adlfs >=2021.10.0
# fsspec==2021.10.1
# ray[default]==1.9.0
ws = Workspace.from_config()
base_conda_dep =['adlfs==2021.10.0','pip']
base_pip_dep = ['ray[tune]==1.9.1', 'xgboost_ray==0.1.5', 'dask==2021.12.0','pyarrow >= 5.0.0','fsspec==2021.10.1', 'torch','torchvision==0.8.1']

compute_cluster = 'worker-cpu-v3'
maxnode =5
vm_size='STANDARD_DS3_V2'
vnet='rayvnet'
subnet='default'
exp ='ray_on_aml_job'
ws_detail = ws.get_details()
ws_rg = ws_detail['id'].split("/")[4]
vnet_rg=None
try:
    ray_cluster = ComputeTarget(workspace=ws, name=compute_cluster)

    print('Found existing cluster, use it.')
except ComputeTargetException:
    if vnet_rg is None:
        vnet_rg = ws_rg
    compute_config = AmlCompute.provisioning_configuration(vm_size=vm_size,
                                                        min_nodes=0, max_nodes=maxnode,
                                                        vnet_resourcegroup_name=vnet_rg,
                                                        vnet_name=vnet,
                                                        subnet_name=subnet)
    ray_cluster = ComputeTarget.create(ws, compute_cluster, compute_config)

    ray_cluster.wait_for_completion(show_output=True)


python_version = ["python="+platform.python_version()]



conda_packages = python_version+base_conda_dep
pip_packages = base_pip_dep 

conda_dep = CondaDependencies()

rayEnv = Environment(name="rayEnv")
# rayEnv = Environment.get(ws, "rayEnv", version=16)
for conda_package in conda_packages:
    conda_dep.add_conda_package(conda_package)

for pip_package in pip_packages:
    conda_dep.add_pip_package(pip_package)

# # Adds dependencies to PythonSection of myenv
rayEnv.python.conda_dependencies=conda_dep

src = ScriptRunConfig(source_directory='job',
                script='aml_job.py',
                environment=rayEnv,
                compute_target=ray_cluster,
                distributed_job_config=PyTorchConfiguration(node_count=maxnode),
                    # arguments = ["--master_ip",master_ip]
                )
run = Experiment(ws, exp).submit(src)

In [ ]:
from azureml.widgets import RunDetails
RunDetails(run).show()


In [ ]:
import requests

from ray import serve

serve.start()


@serve.deployment
def hello(request):
    name = request.query_params["name"]
    return f"Hello {name}!"


hello.deploy()

# Query our endpoint over HTTP.
response = requests.get("http://127.0.0.1:8000/hello?name=serve").text
assert response == "Hello serve!"

In [ ]:
import requests

import ray
from ray import serve

serve.start()


@serve.deployment
class Counter:
    def __init__(self):
        self.count = 0

    def __call__(self, *args):
        self.count += 1
        return {"count": self.count}


# Deploy our class.
Counter.deploy()

# Query our endpoint in two different ways: from HTTP and from Python.
assert requests.get("http://127.0.0.1:8000/Counter").json() == {"count": 1}
assert ray.get(Counter.get_handle().remote()) == {"count": 2}


In [ ]:
from fastapi import FastAPI

app = FastAPI()

@serve.deployment
@serve.ingress(app)
class Counter:
  def __init__(self):
      self.count = 0

  @app.get("/")
  def get(self):
      return {"count": self.count}

  @app.get("/incr")
  def incr(self):
      self.count += 1
      return {"count": self.count}

  @app.get("/decr")
  def decr(self):
      self.count -= 1
      return {"count": self.count}


In [ ]:
Counter.deploy()


In [ ]:
!curl -X GET localhost:8000/Counter/